In [ ]:
import os
import sys

sys.path.append("../")

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import hydra
import numpy as np
import pytorch_lightning as pl
import torch
from omegaconf import OmegaConf

In [ ]:
# set env variable DATA_DIR again because of hydra
from dotenv import load_dotenv

load_dotenv()
os.environ["DATA_DIR"] = os.environ.get("DATA_DIR")

In [ ]:
experiment = "fm_tops.yaml"

In [ ]:
# load everything from experiment config
with hydra.initialize(version_base=None, config_path="../configs/"):
    cfg = hydra.compose(config_name="train.yaml", overrides=[f"experiment={experiment}"])
    # print(OmegaConf.to_yaml(cfg))

In [ ]:
datamodule = hydra.utils.instantiate(cfg.data)
datamodule.setup()

In [ ]:
data = np.array(datamodule.tensor_test)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from tqdm import tqdm

In [ ]:
print(data.shape)

In [ ]:
color: str = ("#E2001A",)
mask_data = np.ma.masked_where(
    data[:, :, 0] == 0,
    data[:, :, 0],
)
mask = np.expand_dims(mask_data, axis=-1)

fig = plt.figure(figsize=(5, 5))
gs = GridSpec(1, 1)
ax = fig.add_subplot(gs[0])
# idx = np.random.randint(len(data))
for idx in tqdm(range(1000)):
    x_plot = data[idx, :, :2]  # .cpu()
    s_plot = np.abs(data[idx, :, 2])  # .cpu())
    s_plot[mask[idx, :, 0] < 0.0] = 0.0

    ax.scatter(*x_plot.T, s=50 * s_plot, color=color, alpha=0.5)

ax.set_xlabel(r"$\eta$")
ax.set_ylabel(r"$\phi$")

ax.set_xlim(-0.3, 0.3)
ax.set_ylim(-0.3, 0.3)
plt.show()

In [ ]:
def plot_single_jets(
    data: np.ndarray,
    color: str = "#E2001A",
    save_folder: str = "logs/",
    save_name: str = "sim_jets",
) -> plt.figure:
    """Create a plot with 16 randomly selected jets from the data.

    Args:
        data (_type_): Data to plot.
        color (str, optional): Color of plotted point cloud. Defaults to "#E2001A".
        save_folder (str, optional): Path to folder where the plot is saved. Defaults to "logs/".
        save_name (str, optional): File_name for saving the plot. Defaults to "sim_jets".
    """
    mask_data = np.ma.masked_where(
        data[:, :, 0] == 0,
        data[:, :, 0],
    )
    mask = np.expand_dims(mask_data, axis=-1)
    fig = plt.figure(figsize=(16, 16))
    gs = GridSpec(4, 4)

    for i in tqdm(range(16)):
        ax = fig.add_subplot(gs[i])

        idx = np.random.randint(len(data))
        x_plot = data[idx, :, :2]  # .cpu()
        s_plot = np.abs(data[idx, :, 2])  # .cpu())
        s_plot[mask[idx, :, 0] < 0.0] = 0.0

        ax.scatter(*x_plot.T, s=5000 * s_plot, color=color, alpha=0.5)

        ax.set_xlabel(r"$\eta$")
        ax.set_ylabel(r"$\phi$")

        ax.set_xlim(-0.3, 0.3)
        ax.set_ylim(-0.3, 0.3)

    plt.tight_layout()

    plt.savefig(f"{save_folder}{save_name}.png", bbox_inches="tight")
    return fig

# Test Sliced Wasserstein Distance

In [ ]:
def swd(data: torch.Tensor, preds: torch.Tensor, n_proj: int = 1024) -> torch.Tensor:
    """Sliced Wassersteini Distance
    Compute the Wasserstein distance between two point clouds.
    Inspired by https://github.com/apple/ml-cvpr2019-swd/blob/master/swd.py#L45

    Args:
        data (torch.Tensor) [batch, n_points, feats]: Ground truth.
        preds (torch.Tensor) [batch, n_points, feats]: Predictions.
        n_proj (int, optional): number of random 1d projections. Defaults to 1024.

    Returns:
        wdist (torch.Tensor) [1]: Wasserstein distance
    """

    b, p, f = data.shape  # [batch,points,feats]
    data, preds = data.float(), preds.float()
    proj = torch.randn(f, n_proj, device=data.device)  # [feats, l]
    print(f"proj: {proj.shape}")
    proj *= torch.rsqrt(torch.sum(torch.square(proj), 0, keepdim=True))
    print(f"proj: {proj.shape}")
    proj = proj.view(1, f, n_proj).expand(b, -1, -1)  # first add dim, then expand to batch dim
    print(f"proj: {proj.shape}")
    p1 = torch.matmul(data, proj)  # shape: [batch, n_points, l]
    print(f"p1: {p1.shape}")
    p2 = torch.matmul(preds, proj)  # shape: [batch, n_points, l]
    print(f"p2: {p2.shape}")
    p1, _ = torch.sort(p1, 1, descending=True)  # point wise sorting
    print(f"p1: {p1.shape}")
    p2, _ = torch.sort(p2, 1, descending=True)
    print(f"p2: {p2.shape}")
    wdist = torch.mean(torch.square(p1 - p2))  # MSE
    return wdist

In [ ]:
tensor1 = torch.tensor(data)
tensor2 = torch.tensor(data) + 0.1
print(tensor1.shape)

In [ ]:
swd_ = swd(tensor1, tensor2)
print(swd_)